# Calls

List the calls

In [5]:
from pymongo import MongoClient

import time

# If jupyter runs on localhost:
# MONGO_HOST="localhost"

# If jupyter runs in docker
MONGO_HOST="host.docker.internal"
MONGO_USER="root"
MONGO_PASSWORD="password"
MONGO_PORT=27017

address = 'mongodb://{user}:{password}@{host}:{port}'.format(
    user= MONGO_USER,
    password= MONGO_PASSWORD,
    host= MONGO_HOST,
    port= MONGO_PORT,
)

client = MongoClient(address)
database=client["observertc-reports"]

# the collection we want to query
reportsDatabase = database.reports

# print('connected to %s' % address)

cursor = reportsDatabase.find( { "type": "CALL_EVENT", "payload.name": { "$in": ["CLIENT_JOINED", "CLIENT_LEFT"] } } )

rooms = {}
for record in cursor:
    callEvent = record["payload"]

    roomId = callEvent["roomId"]
    callId = callEvent["callId"]
    userId = callEvent['userId']
    clientId = callEvent['clientId']
    timestamp = callEvent['timestamp']

    calls = rooms.get(roomId, {})
    clients = calls.get(callId, {})
    client = clients.get(clientId, {
        "clientId": clientId,
        "userId": userId
    })
    
    if callEvent["name"] == "CLIENT_JOINED":
        client.update({ "joined": timestamp })
    else:
        client.update({ "left": timestamp })
    
    clients[clientId] = client
    calls[callId] = clients
    rooms[roomId] = calls

for room in rooms.items():
    roomId, calls = room
    print('\nRoom: %s\n--------' % roomId)
    for call in calls.items():
        callId, clients = call
        print('\nCall %s' % callId)
        for client in clients.values():
            userId, clientId, joined = client["userId"], client["clientId"], client["joined"]
            left = client.get("left", int(time.time() * 1000))
            duration = (left - joined) / (60 * 1000)
            print('\n\tUserId: %s \n\tClientId: %s\n\tJoined: %d\n\tLeft: %d\n\tDuration: %d min' % 
                  (userId, clientId, joined, left, duration))


Room: Pyongyang
--------

Call 9236312f-2697-41e4-82d4-3ff43fd404e8

	UserId: Dumbledore 
	ClientId: 8b2a6420-b596-4005-aabf-e849a992502d
	Joined: 1657197987140
	Left: 1657197999786
	Duration: 0 min

	UserId: Fleur Delacour 
	ClientId: 806b77ce-a81c-4e05-9b50-956ec618d442
	Joined: 1657197994033
	Left: 1657197999786
	Duration: 0 min


# Max concurrent streams through SFU over time

In [6]:
# Match all tracks connected to an sfu
stage_match = {
    "$match": {
        "$or": [{ "type": "INBOUND_VIDEO_TRACK" }, { "type": "INBOUND_AUDIO_TRACK" },
                { "type": "OUTBOUND_VIDEO_TRACK" }, { "type": "OUTBOUND_AUDIO_TRACK" }],
        "$or": [{"payload.sfuStreamId": { "$exists" : True }}, 
                {"payload.sfuSinkId": { "$exists" : True }, "payload.sfuStreamId": { "$exists" : True }}]
   }
}
# group by clients
stage_group = {
   "$group": {
         "_id": {
             "clientId": "$payload.clientId",
             "sfuSinkId": "$payload.sfuSinkId",
             "sfuStreamId": "$payload.sfuStreamId"
          },
           "reportsNum": { "$sum": 1 }, 
   }
}

# the pipeline
pipeline = [
   stage_match, 
   stage_group,
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of concurrent sfu streams", len(list(cursor)))

number of concurrent sfu streams 8


# Number of calls over time

In [42]:
stage_match = {
    "$match": {
        "type": "CALL_EVENT",
        "payload.name": "CALL_STARTED",
   }
}

pipeline = [
   stage_match, 
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of calls", len(list(cursor)))
num = 0

number of calls 3


# % conferences with 2 participants over time

In [46]:
stage_match = {
    "$match": {
        "type": "CALL_EVENT",
        "payload.name": "CLIENT_JOINED",
   }
}
# group by callId
stage_group = {
   "$group": {
         "_id": "$payload.callId",
         "clientsNum": { "$sum": 1 }, 
   }
}

pipeline = [
   stage_match, 
   stage_group,
]

cursor = reportsDatabase.aggregate(pipeline)
total_conferences = 0
p2p_conferences = 0
for report in cursor:
    total_conferences += 1
    p2p_conferences += 1 if report["clientsNum"] == 2 else 0
print("number of conferences have 2 participants", p2p_conferences / total_conferences * 100, "%")

number of conferences have 2 participants 66.66666666666666 %


# Average conference size

In [47]:
stage_match = {
    "$match": {
        "type": "CALL_EVENT",
        "payload.name": "CLIENT_JOINED",
   }
}
# group by callId
stage_group = {
   "$group": {
         "_id": "$payload.callId",
         "clientsNum": { "$sum": 1 }, 
   }
}

# the pipeline
pipeline = [
   stage_match, 
   stage_group,
]

cursor = reportsDatabase.aggregate(pipeline)
total_conferences = 0
nr_of_clients = 0
for report in cursor:
    total_conferences += 1
    nr_of_clients += report["clientsNum"]

print("Average conference size", nr_of_clients / total_conferences)

Average conference size 2.3333333333333335


# Number of unique users (unique cookie ids for a given time period)

In [54]:
stage_match = {
    "$match": {
        "type": "CLIENT_EXTENSION_DATA",
        "payload.type": "USER_FINGERPRINT",
   }
}
# group by callId
stage_group = {
   "$group": {
         "_id": "$payload.payload",
   }
}

# the pipeline
pipeline = [
   stage_match, 
   stage_group,
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of unique users", len(list(cursor)))

number of unique users 0


# % of user joins with media issues over time

In [51]:
cursor = reportsDatabase.find({
    "type": "CALL_EVENT",
    "payload.name": "CLIENT_JOINED"
})
number_of_clients = len(list(cursor))

cursor = reportsDatabase.find({
    "type": "CALL_META_DATA",
    "payload.type": "USER_MEDIA_ERROR"
})

number_of_user_media_errors = len(list(cursor))

print("% of user joins with media issues over time", number_of_user_media_errors / number_of_clients)

% of user joins with media issues over time 0.0


# % of user joins (or join attempts) with connectivity issues over time

In [52]:
cursor = reportsDatabase.find({
    "type": "CALL_EVENT",
    "payload.name": "CLIENT_JOINED"
})
number_of_clients = len(list(cursor))

cursor = reportsDatabase.find({
    "type": "CLIENT_EXTENSION_DATA",
    "payload.type": "ICE_DISCONNECTED"
})

number_of_connectivity_issue = len(list(cursor))

print("% of user joins (or join attempts) with connectivity issues over time", number_of_connectivity_issue / number_of_clients)



% of user joins (or join attempts) with connectivity issues over time 0.0
